In [83]:
# Import libraries

import os
import sys
import glob

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

import numpy as np

import wandb

# Include src directory in path to import custom modules
if '..\\src' not in sys.path:
    sys.path.append('..\\src')

from models.model_unet import UNet3D
from utils.utils import read_hyperparams
from data.datasets import KneeSegDataset3D
from models.evaluation import bce_dice_loss #, dice_coefficient, batch_dice_coeff


In [66]:
sys.path

['c:\\Users\\james\\Documents\\projects\\pred-knee-replacement-oai\\notebooks',
 'c:\\Users\\james\\miniconda3\\envs\\pred-knee-replacement-oai\\python312.zip',
 'c:\\Users\\james\\miniconda3\\envs\\pred-knee-replacement-oai\\DLLs',
 'c:\\Users\\james\\miniconda3\\envs\\pred-knee-replacement-oai\\Lib',
 'c:\\Users\\james\\miniconda3\\envs\\pred-knee-replacement-oai',
 '',
 'c:\\Users\\james\\miniconda3\\envs\\pred-knee-replacement-oai\\Lib\\site-packages',
 'c:\\Users\\james\\miniconda3\\envs\\pred-knee-replacement-oai\\Lib\\site-packages\\win32',
 'c:\\Users\\james\\miniconda3\\envs\\pred-knee-replacement-oai\\Lib\\site-packages\\win32\\lib',
 'c:\\Users\\james\\miniconda3\\envs\\pred-knee-replacement-oai\\Lib\\site-packages\\Pythonwin',
 '..\\src',
 '..\\src',
 '..\\src']

In [2]:
os.getcwd()

'c:\\Users\\james\\Documents\\projects\\pred-knee-replacement-oai\\notebooks'

In [4]:
# Set Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# Read in hyperparams
hyperparams = read_hyperparams('..\src\models\hyperparams_unet.txt')
print(hyperparams)


{'run_name': 'high_lr_40_epoch', 'l_rate': 0.001, 'num_epochs': 40.0, 'batch_size': 4.0, 'threshold': 0.5, 'transforms': 'True'}


<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\james\AppData\Local\Temp\ipykernel_24948\3746911915.py:2: SyntaxWarning: invalid escape sequence '\s'
  hyperparams = read_hyperparams('..\src\models\hyperparams_unet.txt')


In [24]:
# Define data directory
DATA_DIRECTORY = 'C:/Users/james/OneDrive - University of Leeds/1. Projects/1.1 PhD/1.1.1 Project/Data/OAI Subset'
DATA_TRAIN_DIRECTORY = 'C:/Users/james/OneDrive - University of Leeds/1. Projects/1.1 PhD/1.1.1 Project/Data/OAI Subset/train'
DATA_VALID_DIRECTORY = 'C:/Users/james/OneDrive - University of Leeds/1. Projects/1.1 PhD/1.1.1 Project/Data/OAI Subset/valid'

DATA_RAW_DIRECTORY = '../data/raw'
DATA_PROCESSED_DIRECTORY = '../data/processed'
DATA_INTERIM_DIRECTORY = '../data/processed'

In [17]:
os.path.basename('C:/Users/james/OneDrive - University of Leeds/1. Projects/1.1 PhD/1.1.1 Project/Data/OAI Subset/valid/valid/valid_014_V00.seg')

'valid_014_V00.seg'

In [22]:
# Get paths for training and and validation data
# Get the paths

# Return file name from filepath

train_paths = np.array([os.path.basename(i).split('.')[0] for i in glob.glob(f"{DATA_TRAIN_DIRECTORY}/*.im")])
val_paths = np.array([os.path.basename(i).split('.')[0] for i in glob.glob(f"{DATA_VALID_DIRECTORY}/*.im")])

In [10]:
# Set transforms

if hyperparams['transforms'] == "True":
    # Let's try a horizontal flip transform
    transform = transforms.functional.hflip
else:
    transform = None

In [42]:
# Define PyTorch datasets and dataloader

# Define datasets
train_dataset = KneeSegDataset3D(train_paths, DATA_DIRECTORY, transform=transform)
val_dataset = KneeSegDataset3D(val_paths, DATA_DIRECTORY, split='valid')

# Define dataloaders
train_loader = DataLoader(train_dataset, batch_size=int(hyperparams['batch_size']), num_workers = 1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, num_workers = 1, shuffle=False)

In [68]:
a = iter(train_loader)

In [69]:
test = next(a)

In [70]:
test[0].shape

torch.Size([4, 1, 200, 256, 160])

In [71]:
# Create model
model = UNet3D(1, 1, 16)

In [73]:
# Specifiy criterion and optimiser

criterion = bce_dice_loss
l_rate = hyperparams['l_rate']
optimizer = optim.Adam(model.parameters(), lr=l_rate)

In [79]:
# How long to train for?
num_epochs = int(hyperparams['num_epochs'])

In [80]:
# Threshold for predicted segmentation mask
threshold = hyperparams['threshold']
